In [1]:
!pip install spotipy

In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [110]:
import os
from dotenv import load_dotenv

In [112]:
load_dotenv("test.env")

True

In [94]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [96]:
import base64
from requests import post,get
import json


In [114]:

# Obtener el client_id y client_secret
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')


In [116]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET))

In [9]:
playlist_id = "47cZFdxdjGS41YdSqqQlw3"
playlist = sp.playlist(playlist_id)
print(playlist['name'])  # Print the playlist's name
print(playlist['description'])  # Print the playlist's description

200 canciones icónicas de la Historia



In [11]:
tracks = playlist['tracks']['items']

In [118]:
#list[tracks]

In [15]:
track_data = [] 

# Loop through the list items and extract the data we want
for track in tracks:
    track_name = track['track']['name']
    artist_name = track['track']['artists'][0]['name']
    track_id = track['track']['id']
    popularity = track['track']['popularity']
    duration_ms = track['track']['duration_ms']

    # Get the artist ID to fetch the artist genres
    artist_id = track['track']['artists'][0]['id']

    # Get the artist genres using the artist ID and the sp.artist method (Spotipy's artist endpoint)
    artist_info = sp.artist(artist_id) # <- Fetch the artist info
    artist_genres = artist_info['genres'] # <- Extract the genres from the artist info
    artist_genres_str = ', '.join(artist_genres) # <- This is to convert the list of genres to a string

    
    # Append the data to the list as a dictionary, if I append as dictionary, I don't have to specify the columns when creating the DataFrame
    track_data.append({ # <- Dictionary "{}", not list "[]"
        'track_name': track_name, 
        'artist_name': artist_name, 
        'track_id': track_id,
        'popularity': popularity, 
        'duration_ms': duration_ms, 
        'artist_genres': artist_genres_str
    })

# Create a pandas DataFrame from the list of dictionaries. Since I appended dictionary, I don't have to specify the columns
df2 = pd.DataFrame(track_data)
df2.head()

,track_name,artist_name,track_id,popularity,duration_ms,artist_genres
0,Imagine - Remastered 2010,John Lennon,7pKfPomDEeI4TPT6EOYjn9,76,187866,"classic rock, rock"
1,Bohemian Rhapsody,Queen,3z8h0TU7ReDPLIbEnYhWZb,71,354947,"classic rock, glam rock, rock"
2,Hey Jude - Remastered 2009,The Beatles,3m7V717IKZqZLW5qUIOxdD,58,431333,"british invasion, classic rock, merseybeat, ps..."
3,Billie Jean,Michael Jackson,5ChkMS8OtdzJeqyybCc9R5,77,293826,"r&b, soul"
4,(I Can't Get No) Satisfaction - Mono Version,The Rolling Stones,2PzU4IB8Dr6mxV3lHuaG34,76,222813,"album rock, british invasion, classic rock, rock"


In [17]:
df2.to_csv("spotify2dfsinfeatures.csv")

In [31]:
track_id

'102YUQbYmwdBXS7jwamI90'

In [19]:
import time

In [21]:
#function for every class:
features = []

for index, row in df2.iterrows():
    audio_features = sp.audio_features(row["track_id"])[0] #need to change [0]?? El [0] en sp.audio_features(...)[0] es necesario porque el método sp.audio_features de la librería Spotipy devuelve una lista de diccionarios, incluso si solo se solicita una única canción.
    time.sleep(3)
    features.append(audio_features)

In [23]:
df_testfeat = pd.DataFrame(features)

In [25]:
df_testfeat.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.547,0.257,0,-12.358,1,0.0252,0.9070,0.183000,0.0935,0.169,75.752,audio_features,7pKfPomDEeI4TPT6EOYjn9,spotify:track:7pKfPomDEeI4TPT6EOYjn9,https://api.spotify.com/v1/tracks/7pKfPomDEeI4...,https://api.spotify.com/v1/audio-analysis/7pKf...,187867,4
1,0.390,0.395,0,-9.966,0,0.0501,0.2880,0.000000,0.2070,0.247,144.320,audio_features,3z8h0TU7ReDPLIbEnYhWZb,spotify:track:3z8h0TU7ReDPLIbEnYhWZb,https://api.spotify.com/v1/tracks/3z8h0TU7ReDP...,https://api.spotify.com/v1/audio-analysis/3z8h...,354947,4
2,0.399,0.555,10,-10.156,1,0.0259,0.0275,0.000013,0.0928,0.663,145.164,audio_features,3m7V717IKZqZLW5qUIOxdD,spotify:track:3m7V717IKZqZLW5qUIOxdD,https://api.spotify.com/v1/tracks/3m7V717IKZqZ...,https://api.spotify.com/v1/audio-analysis/3m7V...,431333,4
3,0.920,0.654,11,-3.051,0,0.0401,0.0236,0.015300,0.0360,0.847,117.046,audio_features,5ChkMS8OtdzJeqyybCc9R5,spotify:track:5ChkMS8OtdzJeqyybCc9R5,https://api.spotify.com/v1/tracks/5ChkMS8OtdzJ...,https://api.spotify.com/v1/audio-analysis/5Chk...,293827,4
4,0.723,0.863,2,-7.890,1,0.0338,0.0383,0.031700,0.1280,0.931,136.302,audio_features,2PzU4IB8Dr6mxV3lHuaG34,spotify:track:2PzU4IB8Dr6mxV3lHuaG34,https://api.spotify.com/v1/tracks/2PzU4IB8Dr6m...,https://api.spotify.com/v1/audio-analysis/2PzU...,222813,4


In [27]:
df_testfeat.to_csv("spotifyfeatures2.csv")